## ECE 239 Final Project

In this project we explore various deep learning algorithms on the EEG dataset. 


In [7]:
# Imports and setup

import numpy as np
import tensorflow as tf
from tensorflow import keras

from utils.load_data import *
from models.cnn import CNN

%load_ext autoreload
%autoreload 2

In [8]:
# Loads data from the EEG dataset and removes all EOG data

person_train_valid, X_train_valid, y_train_valid, person_test, X_test, y_test = load_EEG_data()

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


## Split into train and validation datasets

In [9]:
X_train, y_train, X_val, y_val = split_train_val(X_train_valid, y_train_valid, percent_validation=0.1)

print ('Training data shape: {}'.format(X_train.shape))
print ('Training target shape: {}'.format(y_train.shape))
print ('Validation data shape: {}'.format(X_val.shape))
print ('Validation target shape: {}'.format(y_val.shape))

Training data shape: (1904, 22, 1000)
Training target shape: (1904,)
Validation data shape: (211, 22, 1000)
Validation target shape: (211,)


## One hot encoding for all labels

In [10]:
print (y_val[33])
print (y_val.shape)
print (y_test.shape)

def convert_to_label(num):
    return np.array([1.0*(num==769),  1.0*(num==770), 1.0*(num==771), 1.0*(num==772)])

y_train_labels = np.array([convert_to_label(yi) for yi in y_train])
y_val_labels = np.array([convert_to_label(yi) for yi in y_val])
y_test_labels = np.array([convert_to_label(yi) for yi in y_test])

print (y_val_labels[33])
print (y_val_labels.shape)
print (y_test_labels.shape)


771
(211,)
(443,)
[0. 0. 1. 0.]
(211, 4)
(443, 4)


## Classifying data using a CNN

In [11]:
cnn = CNN(X_train, y_train_labels, X_val, y_val_labels, X_test, y_test_labels)
cnn.train(stride=2, optimizer='adam', epochs=5)
test_loss, test_accuracy = cnn.evaluate()
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Train on 1904 samples, validate on 211 samples
Epoch 1/5
1904/1904 [==============================] - 1s 428us/sample - loss: 1.4199 - acc: 0.2773 - val_loss: 1.4022 - val_acc: 0.2701
Epoch 2/5
1904/1904 [==============================] - 0s 205us/sample - loss: 1.3731 - acc: 0.3151 - val_loss: 1.4088 - val_acc: 0.3175
Epoch 3/5
1904/1904 [==============================] - 0s 230us/sample - loss: 1.3402 - acc: 0.3414 - val_loss: 1.4041 - val_acc: 0.2938
Epoch 4/5
1904/1904 [==============================] - 0s 197us/sample - loss: 1.3122 - acc: 0.3682 - val_loss: 1.3918 - val_acc: 0.2938
Epoch 5/5
443/443 [==============================] - 0s 106us/sample - loss: 1.4507 - acc: 0.2867
Test loss: 1.450662408940679
Test accuracy: 0.2866817
